# Mount Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Library installion

In [ ]:
!pip install openai

In [ ]:
!pip install tiktoken

# Library Imports

In [ ]:
import openai
import os
import re
import ast

In [ ]:
import numpy as np
import openai
import pandas as pd
import pickle
import tiktoken

COMPLETIONS_MODEL = "text-davinci-003"
EMBEDDING_MODEL = "dialog_research_text_embedding_002"

# Env Defining

In [ ]:
os.environ["OPENAI_API_KEY"] = "d60151e62c8e4a4dbabc7239930b6ce5"
openai.api_type = "azure"
openai.api_base = "https://genai-research-dialog.openai.azure.com/"
openai.api_version = "2023-03-15-preview"
openai.api_key = os.getenv("OPENAI_API_KEY")

In [ ]:
# pip install tiktoken

# Import Data

In [ ]:
df_process = pd.read_csv("/content/drive/MyDrive/GP/Procuments (1).csv")
df_process = df_process.set_index(["concat"])

In [ ]:
document_embeddings_process = pd.read_csv("/content/drive/MyDrive/GP/procumentEM (1).csv")
document_embeddings1 = document_embeddings_process.drop(document_embeddings_process.columns[0], axis=1)
document_embeddings_T1 = document_embeddings1.transpose()
document_embeddings_T1.columns = list(df_process.index)

In [ ]:
df_general = pd.read_csv("/content/drive/MyDrive/GP/notes.csv")
df_general = df_general.set_index(["content"])

In [ ]:
document_embeddings_general = pd.read_csv("/content/drive/MyDrive/GP/embeddings1 (1).csv")
document_embeddings2 = document_embeddings_general
document_embeddings_T2 = document_embeddings2.transpose()
document_embeddings_T2.columns = list(df_general.index)

# Embeddings Functions

for process


In [ ]:
def get_embedding(text, model = EMBEDDING_MODEL):
  result = openai.Embedding.create(
      deployment_id=model,
      input=text
  )
  return result["data"][0]["embedding"]

In [ ]:
def compute_doc_embeddings(df_process):
    """
    Create an embedding for each row in the dataframe using the OpenAI Embeddings API.

    Return a dictionary that maps between each embedding vector and the index of the row that it corresponds to.
    """
    return {
        idx: get_embedding(r.concat) for idx, r in df_process.iterrows()
    }
compute_doc_embeddings(df_process)

In [ ]:
def load_embeddings(fname):
    """
    Read the document embeddings and their keys from a CSV.

    fname is the path to a CSV with exactly these named columns:
        "Heading 1", "Heading 2", "Heading 3", "concat", "Content" ,"0", "1", ... up to the length of the embedding vectors.
    """

    df_process = pd.read_csv(fname, header=0)
    max_dim = max([int(c) for c in df_process.columns if c != "concat"])
    return {
        r.concat1: [r[str(i)] for i in range(max_dim + 1)] for _, r in df_process.iterrows()
    }

Embeddings for general

In [ ]:
def get_embedding(text, model = EMBEDDING_MODEL):
  result = openai.Embedding.create(
      deployment_id=model,
      input=text
  )
  return result["data"][0]["embedding"]

In [ ]:
def compute_doc_embeddings(df_general):
    """
    Create an embedding for each row in the dataframe using the OpenAI Embeddings API.

    Return a dictionary that maps between each embedding vector and the index of the row that it corresponds to.
    """
    return {
        idx: get_embedding(r['content']) for idx, r in df_general.iterrows()
    }

In [ ]:
def load_embeddings(f1name):
    """
    Read the document embeddings and their keys from a CSV.

    fname is the path to a CSV with exactly these named columns:
        "context, "Content".. up to the length of the embedding vectors.
    """

    df_general = pd.read_csv(f1name, header=0)
    max_dim = max([int(c) for c in df_general.columns if c != "content"])
    return {
        r.content1: [r[str(i)] for i in range(max_dim + 1)] for _, r in df_general.iterrows()
    }

In [ ]:
def completions_function(prompt):
  response = openai.ChatCompletion.create(
        engine="dialog_research_gpt35turbo",
        temperature = 0,
        messages=[
        {"role": "user", "content": prompt}
    ]
    )
  return response['choices'][0]['message']['content']


In [ ]:
# document_embeddings_T2

# Similarity Checking Functions

In [ ]:
def vector_similarity(x, y):
    """
    Returns the similarity between two vectors.

    Because OpenAI Embeddings are normalized to length 1, the cosine similarity is the same as the dot product.
    """
    return np.dot(np.array(x), np.array(y))

def order_document_sections_by_query_similarity(query, context):
    """
    Find the query embedding for the supplied query, and compare it against all of the pre-calculated document embeddings
    to find the most relevant sections.

    Return the list of document sections, sorted by relevance in descending order.
    """

    query_embedding = get_embedding(query)

    document_similarities = sorted([
        (vector_similarity(query_embedding, doc_embedding), doc_index) for doc_index, doc_embedding in context.items()
    ], reverse=True)

    return document_similarities

In [ ]:
def cosine_similarity(x, y):
    """
    Returns the cosine similarity between two vectors.
    """
    return np.dot(np.array(x), np.array(y)) / (np.linalg.norm(np.array(x)) * np.linalg.norm(np.array(y)))

def order_document_sections_by_query_similarity(query, context):
    """
    Find the query embedding for the supplied query, and compare it against all of the pre-calculated document embeddings
    to find the most relevant sections.

    Return the list of document sections, sorted by relevance in descending order.
    """
    query_embedding = get_embedding(query)

    document_similarities = sorted([
        (cosine_similarity(query_embedding, doc_embedding), doc_index) for doc_index, doc_embedding in context.items()
    ], reverse=True)

    return document_similarities


Genaral

In [ ]:
def cosine_similarity(x, y):
    """
    Returns the cosine similarity between two vectors.
    """
    return np.dot(np.array(x), np.array(y)) / (np.linalg.norm(np.array(x)) * np.linalg.norm(np.array(y)))

def order_document_sections_by_query_similarity(query, content):
    """
    Find the query embedding for the supplied query, and compare it against all of the pre-calculated document embeddings
    to find the most relevant sections.

    Return the list of document sections, sorted by relevance in descending order.
    """
    query_embedding = get_embedding(query)

    document_similarities = sorted([
        (cosine_similarity(query_embedding, doc_embedding), doc_index) for doc_index, doc_embedding in content.items()
    ], reverse=True)

    return document_similarities

In [ ]:
def vector_similarity(x, y):
    """
    Returns the similarity between two vectors.

    Because OpenAI Embeddings are normalized to length 1, the cosine similarity is the same as the dot product.
    """
    return np.dot(np.array(x), np.array(y))

def order_document_sections_by_query_similarity(query, content):
    """
    Find the query embedding for the supplied query, and compare it against all of the pre-calculated document embeddings
    to find the most relevant sections.

    Return the list of document sections, sorted by relevance in descending order.
    """

    query_embedding = get_embedding(query)

    document_similarities = sorted([
        (vector_similarity(query_embedding, doc_embedding), doc_index) for doc_index, doc_embedding in content.items()
    ], reverse=True)

    return document_similarities

In [ ]:
 question = "What is the flow to get budgeted capex LKR 230000?"

In [ ]:
def get_category(question):
  prompt = """
  You are given a question from an employee at 'DIALOG AXIATA'. Your task is to determine whether the question falls into the 'process' or 'general' category.

  Type 1: Process - The question is related to approval process, recommendation process, steps, flow or processes within the limits of authorities.
  Type 2: General - The question is not related to approval process, recommendation process, or processes within the limits of authorities.

  Please identify the type of the question.

  Example Process Questions:
  identify
  1. What is the approval process for expenses?
  2. How do I request a recommendation letter?


  Example General Questions:
  1. What is CXO can do?
  2. How can I improve my productivity?
  3. what is NOTE:10?
  4. Who does CXO can reports to?

  Question: {question}
  Category: {category}
  """
  # Specify the question for classification
  user_question = question

  # Combine the prompt and user question
  combined_prompt = prompt.format(question=user_question, category="")

  # Call the OpenAI API to get the category prediction
  response = openai.Completion.create(
      engine= "dialog_text_davinci_003", # You can adjust the model
      prompt=combined_prompt,
      max_tokens=100  # You can adjust the token limit based on your model's capabilities
  )

  # Extract the category prediction from the response
  category_prediction = response.choices[0].text.strip()

  # print("Category Prediction:", category_prediction)  return category_prediction
  return category_prediction

In [ ]:
def get_similar_set(question):

  category = get_category(question)
  print('category in get_similar_set :', category)

  if (category == "Process"):
    document_embeddingsT1 = document_embeddings_T1
    ss = order_document_sections_by_query_similarity(question, document_embeddingsT1)[:10]

  if (category == "General"):
    document_embeddingsT2 = document_embeddings_T2
    ss = order_document_sections_by_query_similarity(question, document_embeddingsT2)[:4]
  return ss, category

In [ ]:
def get_general_question_context(ss):
  related_text = ''
  for item in range(len(ss)):
    related_text = related_text + ss[item][1] + '\n' + '\n'
  return related_text

In [ ]:
def give_general_answer(ss,query_context_general,question,model):
  prompt = f"""You will be provided with question and the context delimited by triple backticks.
  Your task is to
  1. understand the question and context clearly.
  2. Use the context to answer the question.
  Question:```{question}```

  Context= ```{query_context_general}```
  """
  if(model=='chat'):
    response = openai.ChatCompletion.create(
        engine="dialog_research_gpt35turbo",
        temperature = 0,
        messages=[
        {"role": "user", "content": prompt}
    ]
    )
    return response['choices'][0]['message']['content']

In [ ]:
loa_context = order_document_sections_by_query_similarity(question, document_embeddings_T1)[:10]
query_context = ''
index = []
for i in range(len(loa_context)):
  query_context = query_context  + loa_context[i][1] + '\n'
  index.append(loa_context[i][1])
print(query_context)

In [ ]:
def give_general_answer(ss,query_context_general,question,model):
  prompt = f"""You will be provided with question and the context delimited by triple backticks.
  Your task is to
  1. understand the question and context clearly.
  2. Use the context to answer the question.
  Question:```{question}```

  Context= ```{query_context_general}```
  """
  if(model=='chat'):
    response = openai.ChatCompletion.create(
        engine="dialog_research_gpt35turbo",
        temperature = 0,
        messages=[
        {"role": "user", "content": prompt}
    ]
    )
    return response['choices'][0]['message']['content']

In [ ]:
def give_best(query_context,question,model):
  prompt = f"""You will be provided with a Question and 10 contexts delimited by triple backticks.

  Your task is to follow the steps below.
  Step 1. First understand any value mentioned in Question.These numbers may be mostly associated with a currency LKR or USD. \
          Also there can be suffixes associated with a number. For example LKR 2 million or 2 Mn can be valued as 2000000. Identify only the value in the Question.\
          First write value as number.
  Step 2. Then identify the lower range and upper range for each context. For a example for a Value between LKR 10000000 and equivalent of LKR 25000000, the lower bound is LKR 10000000 and upper bound is LKR 25000000
  Step 3. Then undertand numbers or number ranges in the 10 contexts, from Context 0 to Context 9. These numbers may be mostly associated with a currency LKR or USD. \
          Also there can be suffixes associated with a number. Carefully identify the value of those numbers. For example 2 million or 2 Mn can be valued as 2000000.
  Step 4. Then identify the lower range and upper range for each context. For a example for a Value between LKR 10000000 and equivalent of LKR 25000000, the lower bound is LKR 10000000 and upper bound is LKR 25000000\
          Another example is for a Value greater than LKR 45000000, the lower bound is LKR 45000000 and there is no upper bound. If  there is no lower bound , assign lower bound as 0. If there is no upper bound as LKR 5000000000000.\
          output the lower bound and upper bound in a tuple.
  Step 5. Give the output of Step 1 and Step 4 as a python Dictionary. Keys : Question Value, Context 1, Context 2 ......


  Question:```{question}```

  Context 0= ```{query_context[0][1]}```,
  Context 1= ```{query_context[1][1]}```,
  Context 2= ```{query_context[2][1]}```,
  Context 3= ```{query_context[3][1]}```,
  Context 4= ```{query_context[4][1]}```,
  Context 5= ```{query_context[5][1]}```,
  Context 6= ```{query_context[6][1]}```,
  Context 7= ```{query_context[7][1]}```,
  Context 8= ```{query_context[8][1]}```,
  Context 9= ```{query_context[9][1]}```

  Return the output of Step 4 Only

  """

  if(model=='chat'):
    response = openai.ChatCompletion.create(
        engine="dialog_research_gpt35turbo",
        temperature = 0,
        messages=[
        {"role": "user", "content": prompt}
    ]
    )
    return response['choices'][0]['message']['content']


  if model == 'davinci':
    response = openai.Completion.create(
      engine="dialog_text_davinci_003",
      #model="gpt-35-turbo",
      prompt = prompt,
      temperature=0,
      max_tokens=2000
      )
    return response.choices[0].text

In [ ]:
def give_steps_from_algo(highest_similar_content):
  segments = highest_similar_content.split(' ')
  results = []
  current_segment = ''

  for segment in segments:
      if segment.startswith(('.')):
          if current_segment:
              results.append(current_segment.strip())
          current_segment = segment
      else:
          current_segment += ' ' + segment

  if current_segment:
    results.append(current_segment.strip())

  output = '\n'.join(results)
  return output


In [ ]:
ss, category = get_similar_set(question)
model = 'chat'

print('Question:', question)
print('Category:', category)

if (category == "Process"):
  number_boundaries = give_best(ss,question,model)
  number_boundaries_dict = ast.literal_eval(number_boundaries)
  print(number_boundaries_dict)
  # highest_similar_content = get_LOA(ss,question,df_process,model)

In [ ]:
matching_indexes = []
Question_Set = list()
Binary_list = []
sentences_list = []

for x, y in number_boundaries_dict.items():
    if x == 'Question Value':
        Question_Value = str(y)
    else:
        upper_bound = number_boundaries_dict[x][1]
        lower_bound = number_boundaries_dict[x][0]
        prompt_lower = "Is value " + Question_Value + " lesser than value " + str(lower_bound) + " ? Return TRUE or FALSE only"
        prompt_upper = "Is value " + Question_Value + " greater than value " + str(upper_bound) + " ? Return TRUE or FALSE only"
        tuple_string = (prompt_lower, prompt_upper)
        Question_Set.append(tuple_string)
        L = completions_function(prompt_lower)
        H = completions_function(prompt_upper)
        Binary_list.append((L, H))
        if L.__contains__('FALSE') and H.__contains__('FALSE'):
            matching_indexes.append(len(Binary_list) - 1)

# Extract indexes
indexes = matching_indexes

for index in matching_indexes:
    if 0 <= index < len(Question_Set):
        print(ss[index][1])
        context_matched = ss[index]
        for sentence in context_matched:
            sentences_list.append(sentence)

# Output indexes and sentences_list
print("Indexes:", indexes)
print("Sentences List:", sentences_list)

In [ ]:
selected_items = sentences_list[1::2]  # Start at index 1, step by 2
print(selected_items)


In [ ]:
# Strings to remove (modify as needed)
strings_to_remove = ['LKR', 'Million', 'Rupees','Mn','million']

# Function to remove numbers and specified substrings from a sentence
def remove_numbers_and_strings(sentence, strings_to_remove):
    # Remove numbers
    sentence_without_numbers = re.sub(r'\d+', '', sentence)

    # Remove specified substrings
    for string in strings_to_remove:
        sentence_without_numbers = sentence_without_numbers.replace(string, '')

    return sentence_without_numbers.strip()  # Remove leading/trailing whitespace

# # Remove numbers and specified substrings from all sentences in the list
cleaned_sentences = [remove_numbers_and_strings(sentence, strings_to_remove) for sentence in selected_items]

# Print the cleaned sentences as a list
print(cleaned_sentences)


sentence_dict = {}

for sentence in selected_items:
    cleaned_sentence = remove_numbers_and_strings(sentence, strings_to_remove)
    sentence_dict[cleaned_sentence] = sentence

# Print the dictionary
for key, value in sentence_dict.items():
    print(f"Sentence without numbers: {key}")
    print(f"Original sentence: {value}")
    print()


In [ ]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Tokenize and vectorize the question and phrases
vectorizer = TfidfVectorizer()
question_vector = vectorizer.fit_transform([question])
phrase_vectors = vectorizer.transform(cleaned_sentences)

# Calculate cosine similarity between the question and each phrase
similarities = cosine_similarity(question_vector, phrase_vectors)

# Find the index of the most similar phrase
most_similar_index = np.argmax(similarities)
most_similar_phrase = cleaned_sentences[most_similar_index]

# Print the most similar phrase and its similarity score
print("Question:", question)
print("Most Similar Phrase:", most_similar_phrase)
print("Cosine Similarity Score:", similarities[0, most_similar_index])


In [ ]:
sentence_dict[most_similar_phrase]
index_ = sentence_dict[most_similar_phrase]
print("question : ", question)
print("most similar phrase : " ,sentence_dict[most_similar_phrase])
similar_content = df_process["Content"][index_]
similar_content

In [ ]:
def final_output (similar_content):
  print("question : ", question)
  print("most similar phrase : " ,sentence_dict[most_similar_phrase])
  similar_content = df_process["Content"][index_]
similar_content

In [ ]:
def get_similar_set(question):

  category = get_category(question)
  print('category in get_similar_set :', category)

  if (category == "Process"):
    document_embeddingsT1 = document_embeddings_T1
    ss = order_document_sections_by_query_similarity(question, document_embeddingsT1)[:10]

  if (category == "General"):
    document_embeddingsT2 = document_embeddings_T2
    ss = order_document_sections_by_query_similarity(question, document_embeddingsT2)[:4]
  return ss, category

In [ ]:
def get_gen(question,df_general,model):
  data = {"Context 0" : ss[0][1],
          "Context 1" : ss[1][1],
          "Context 2" : ss[2][1],
          "Context 3" : ss[3][1],}
  context = give_general_answer(ss,query_context,question,model)
  context = context.strip()
  print(context)
  contexts = re.findall(r"(Context [0-3])", context)
  print(contexts)
  index = data[contexts[0]]
  similar_content = df_general["content"][index]
  return similar_content

In [ ]:
def loa_bot(ss,question, df_process, model):
  ss, category = get_similar_set(question)
  print('Question:', question)
  print('Category:', category)

  if (category == "Process"):
    highest_similar_content =df_process["Content"][index_]
    steps = give_steps_from_algo(highest_similar_content)
    return steps

  if (category == "General"):
    query_context_general = get_general_question_context(ss)
    # print("context:",query_context_general)
    answer = give_general_answer(ss,query_context_general,question,model)
    return answer

In [ ]:
loa_bot(ss,question, df_process, 'chat')

Evaluation


In [ ]:
def get_completion_from_messages(messages,
                                 model="gpt-3.5-turbo",
                                 engine="dialog_research_gpt35turbo",
                                 temperature=0,
                                 max_tokens=1000):
    import openai
    import os


    # Access the environment variables
    openai.api_key = "d60151e62c8e4a4dbabc7239930b6ce5"
    openai.api_type = "azure"
    openai.api_base = "https://genai-research-dialog.openai.azure.com/"
    openai.api_version = "2023-03-15-preview"



    response = openai.ChatCompletion.create(
        model=model,
        engine=engine,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens,
    )
    return [response.choices[0].message["content"],response.usage]

In [ ]:
def evaluation_prompt(question:str,  ideal_answer:str, bot_answer:str)->str:
    print("ideal answer, bot answer",ideal_answer, bot_answer)

    system_message = """\
    You are an assistant that evaluates how well the customer service agent \
    answers a user question by comparing the response to the ideal (expert) response
    Output a single letter and nothing else.
    """

    user_message = f"""\
You are comparing a submitted answer to an expert answer on a given question. Here is the data:
    [BEGIN DATA]
    ************
    [Question]: {question}
    ************
    [Expert]: {ideal_answer}
    ************
    [Submission]: {bot_answer}
    ************
    [END DATA]

Compare the factual content of the submitted answer with the expert answer. Ignore any differences in style, grammar, or punctuation.
    The submitted answer may either be a subset or superset of the expert answer, or it may conflict with it. Determine which case applies. Answer the question by selecting one of the following options:
    (1) The submitted answer is either a subset, superset or same as the expert answer and is fully consistent with it.
    (0) There is a disagreement between the submitted answer and the expert answer.
  choice_strings: 0, 1
"""

    messages = [
        {'role': 'system', 'content': system_message},
        {'role': 'user', 'content': user_message}
    ]

    response = get_completion_from_messages(messages)
    return response

In [ ]:
def loa_bot_evaluation(question, df_process, model):
  ss , category = get_similar_set(question)


  if (category == "Process"):

    data = {"Context 0" : ss[0][1],
          "Context 1" : ss[1][1],
          "Context 2" : ss[2][1],
          "Context 3" : ss[3][1],
          "Context 4" : ss[4][1],
          "Context 5" : ss[5][1],
          "Context 6" : ss[6][1],
          "Context 7" : ss[7][1],
          "Context 8" : ss[8][1],
          "Context 9" : ss[9][1],}

    context = give_best(ss, question,'chat')
    context = context.strip()
    contexts = re.findall(r"(Context\s[0-9])", context)

    index = data[contexts[0]]
    print("index",index)
    return index

  else :
    print("error")


Evaluation


In [ ]:
import json
import csv
# Placeholder CSV content for demonstration purposes
def func_eval_prev(input_file:str, output_file:str ,df_process , model = 'chat'):
    with open(input_file, 'r') as csvfile:
        reader = csv.reader(csvfile)
        # Skip header
        next(reader)
        csv_content = list(reader)

    results = []
    for row in csv_content:
        user_input = row[0]
        ideal_answer = row[1]

        response = loa_bot_evaluation(user_input , df_process, model)
        output = response[0]
        usage = response[1]

        evaluation_result = evaluation_prompt(user_input, ideal_answer,response)

        results.append([user_input,ideal_answer, response, evaluation_result[0]])

    # Write results to a new CSV
    with open(output_file, "w", newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(["User Input","Ideal Answer", "Bot Answer", "Evaluation Result"])
        writer.writerows(results)

input_file = "/content/drive/MyDrive/GP/evaluation.csv"
output_file = "/content/drive/MyDrive/GP/evaluation_result.csv"


# Process the CSV
func_eval_prev(input_file, output_file,df_process,'chat')

output_file